In [1]:
import torch

In [29]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [47]:
import nlpaug.augmenter.word as naw

text = 'The quick brown fox jumped over the lazy dog'
back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de', 
    to_model_name='facebook/wmt19-de-en',
    device=device,batch_size=200,verbose=1
)


In [68]:
ret = torch.load("./text_embed_w_btaug.bin")

In [69]:
ret[list(ret.keys())[0]]

[['Closeup of bins of food that include broccoli and bread.',
  tensor([[-0.3884,  0.0229, -0.0522,  ..., -0.4899, -0.3066,  0.0675],
          [-0.3273,  0.8241,  1.2678,  ...,  0.1957, -0.4083, -1.0814],
          [-0.1421,  0.2648,  1.3094,  ...,  0.1651, -1.1205, -0.4609],
          ...,
          [-0.7959,  0.3240, -0.3614,  ..., -2.4090, -0.4159,  0.4823],
          [-0.1493, -0.4688, -2.0566,  ..., -0.4839, -0.5929, -0.7875],
          [-0.2017, -0.4129, -2.0257,  ..., -0.6526, -0.6964, -0.7721]])],
 ['Close-up of food containers with broccoli and bread.',
  tensor([[-0.3884,  0.0229, -0.0522,  ..., -0.4899, -0.3066,  0.0675],
          [ 0.6995, -0.6628,  0.9205,  ...,  1.0396,  1.6607,  0.0387],
          [ 2.8400,  0.0895,  1.6586,  ..., -0.2983,  0.0955,  0.3390],
          ...,
          [ 0.4123,  0.0966,  0.1883,  ..., -1.2672,  0.6310, -0.1029],
          [ 0.5071, -1.2406, -0.5554,  ...,  1.0425, -0.6568, -0.8122],
          [ 0.2931, -1.0535, -0.7505,  ...,  0.8049, -0

In [40]:
bt_aug_data = []
original_data = []

with open("./metadata.jsonl") as fin:
    for line in tqdm(fin.readlines()):
        tmp=json.loads(line)
        original_data.append([tmp['file_name'],tmp['text']])

100%|███████████████████████████████████████| 50026/50026 [00:00<00:00, 130190.98it/s]


In [50]:
chunk_size = 200

In [54]:
all_text = [i[1] for i in original_data]
aug_text = []
for i in tqdm([all_text[i:i + chunk_size] for i in range(0, len(all_text), chunk_size)]):
    aug_text = aug_text + back_translation_aug.augment(i)

100%|█████████████████████████████████| 251/251 [23:39<00:00,  5.65s/it]


In [93]:
swap = naw.RandomWordAug(action="swap")
augmented_text = swap.augment(text)
crop = naw.RandomWordAug(action='crop')
augmented_text = crop.augment(augmented_text)

In [95]:
aug_text_bad = []
for i in tqdm([all_text[i:i + chunk_size] for i in range(0, len(all_text), chunk_size)]):
    aug_text_bad = aug_text_bad + crop.augment(swap.augment(i))

100%|██████████████████████████████████████████████████| 251/251 [00:06<00:00, 36.18it/s]


In [96]:
aug_text_dict_rand = {}
for i,j in enumerate(original_data):
    aug_text_dict_rand[j[0]] = aug_text_bad[i]

In [97]:
aug_text_dict_rand

{'000000000009_0.jpg': 'Of food that broccoli and. bread',
 '000000000009_1.jpg': 'Meal is presented in brightly colored plastic.',
 '000000000009_2.jpg': 'there are filled containers different of',
 '000000000009_3.jpg': 'Dishes meat holding, , vegetables fruit, and',
 '000000000009_4.jpg': 'Bunch a trays of food.',
 '000000000025_0.jpg': 'A giraffe food of tree the.',
 '000000000025_1.jpg': 'A up standing a nearby',
 '000000000025_2.jpg': 'A giraffe mother with. the forest',
 '000000000025_3.jpg': 'Giraffes two standing in filled.',
 '000000000025_4.jpg': 'Giraffe standing next a to with forest',
 '000000000030_0.jpg': 'Flower is vase on a sitting porch stand',
 '000000000030_1.jpg': 'Vase white with colored of it.',
 '000000000030_2.jpg': 'a vase white flowers a stage',
 '000000000030_3.jpg': 'A vase filled flowers colored.',
 '000000000030_4.jpg': 'With white outside flowers on a sunny. day',
 '000000000034_0.jpg': 'Zebra grazing a on lush a.',
 '000000000034_1.jpg': 'Zebra reachin

In [98]:
with open("metadata_with_crop_swap.json","w") as f:
    json.dump(aug_text_dict_rand,f)

In [86]:
with open("metadata_with_bt.json","w") as f:
    json.dump(aug_text_dict,f)

In [63]:
aug_text_dict = {}
for i,j in enumerate(original_data):
    aug_text_dict[j[0]] = aug_text[i]

In [67]:
from transformers import  CLIPTextModel, CLIPTokenizer
import torch
import json
from tqdm import tqdm
from diffusers import StableDiffusionPipeline
batch_size=32
device = torch.device("cuda:1")

model_name = "CompVis/stable-diffusion-v1-4"

# pipe = StableDiffusionPipeline.from_pretrained(model_name, torch_dtype=torch.float16, revision="fp16")
# pipe = pipe.to(device)

tokenizer = CLIPTokenizer.from_pretrained(model_name, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(model_name, subfolder="text_encoder")

result={}
last_name=""
with open("./metadata.jsonl") as fin:
    for line in tqdm(fin.readlines()):
        tmp=json.loads(line)
        img_name=tmp["file_name"].split("_")[0]+tmp["file_name"].split("_")[1][1:]
        
        origin_name = tmp["file_name"]
        if img_name == last_name:
            result[img_name].append([tmp["text"]])
            result[img_name].append([aug_text_dict[origin_name]])
            
        else:
            result[img_name]=[[tmp["text"]]]
            result[img_name].append([aug_text_dict[origin_name]])
            last_name = img_name

text_encoder.requires_grad_(False)
text_encoder.to(device)
dic=[]

for k in tqdm(result.keys()):
    texts = [text[0] for text in result[k]]
    inputs = tokenizer(texts, max_length=60, padding=True,return_tensors="pt")
    embeddings = text_encoder(inputs["input_ids"].to(device))
    embeddings = embeddings[0].cpu()

    for i in range(len(result[k])):
        result[k][i].append(embeddings[i])


torch.save(result,"./text_embed_w_btaug.bin")
# result=torch.load("./text_embed.bin")
# print(len(result))

    

100%|████████████████████████████| 10000/10000 [01:36<00:00, 103.77it/s]


In [74]:
embeddings = text_encoder(inputs["input_ids"].to(device))